In [70]:
import re
import requests
import pandas
from lxml import etree
from datetime import datetime,timedelta
import time

# 认识天天基金网的数据接口

参考：

https://blog.csdn.net/yys1164922014/article/details/100098533?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.channel_param&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.channel_param

## 基金代码和基金名称

这是所有基金代码、基金名称简称的集合http://fund.eastmoney.com/js/fundcode_search.js

数据量是接近9000条数据。这个可以简单的用个正则表达，re.findall("(.*?)",url_text)匹配一下存在电脑上。然后再弄成excel文件。比较简单。

In [40]:
url = 'http://fund.eastmoney.com/js/fundcode_search.js'
len_per_fund = 5 # 每个基金的信息有5个字段

In [47]:
response = requests.get(url)
url_text = response.content.decode('utf-8')
url_text_list = re.findall('\"(.*?)\"',url_text)
num5 = int(len(url_text_list)/5)

In [48]:
num5

13965

In [42]:
url_text_list[0:5]

['000001', 'HXCZHH', '华夏成长混合', '混合型-偏股', 'HUAXIACHENGZHANGHUNHE']

In [49]:
with open("基金列表.txt","w") as f:
    
    wlines = []
    for i in range(num5):
        wline = '\t'.join(url_text_list[i*len_per_fund:(i+1)*len_per_fund])
        wlines.append(wline)
    wlines = '\n'.join(wlines)
    f.write(wlines)

## 基金历史净值接口

url=‘http://fund.eastmoney.com/f10/F10DataApi.aspx?type=lsjz&code=110022&page=10&sdate=2019-01-01&edate=2019-02-13&per=1’
参数情况一目了然。

url变量：

per=1值代表每页的数据量。最多可以有二十行，per=20。设置更高也没用。

code：基金代码

page：当前是第几页，从1开始到pages为止

sdate：开始日期，在最后一页

edate：结束日期，在第一页

显示变量：

records: 在当前条件下，一共有多少条记录

pages：一共有多少页

curpage:当前页码，与url中的page相同


为了代码更加简洁，我设置的每页数据量是1条。这样就更辛苦电脑了。
由于我们这要爬取全部的历史净值，所以要查询到每只基金的起始日期。

var apidata={ content:"
净值日期	单位净值	累计净值	日增长率	申购状态	赎回状态	分红送配

2021-07-19	1.0295	1.0295	-0.38%	场内买入	场内卖出	
2021-07-16	1.0334	1.0334	-0.03%	场内买入	场内卖出	
",records:414,pages:207,curpage:21};

## 查询基金起始日期

http://fund.eastmoney.com/000001.html?spm=search

自己更改中间的基金代码就好
问题是并不是所有基金都能查到起始日期。所以在用re.findall匹配到起止日期的时候，我又加了个IF语句。以防读取不到，产生错误。

In [51]:
def get_start_date(fund_id):
    # 返回日期字符串
    search_url='http://fund.eastmoney.com/'+fund_id+'.html?spm=search'
    # 获取网页内容；设置网页编码，获取起始日期
    response = requests.get(search_url)
    text = response.content.decode('utf-8')
    
    start_date = re.findall('<td><span class="letterSpace01">成 立 日</span>：(.*?)</td>', text)
    # 对无记录的基金代码进行记录，方便后续清除
    if start_date == []:
        start_date = ['无记录'] 
    return start_date[0] # 如果信息正常就会返回开始日期，如果有错误，就会返回无记录。都是字符串的结果

## 根据基金的每一条数据接口的网址，把那条信息抓下来，返回list结果

一条网址的信息是这样：

var apidata={ content:"
净值日期	单位净值	累计净值	日增长率	申购状态	赎回状态	分红送配
2021-07-16	1.0334	1.0334	-0.03%	场内买入	场内卖出	
",records:414,pages:207,curpage:21};

In [87]:
response = requests.get('https://fundf10.eastmoney.com/F10DataApi.aspx?type=lsjz&code=512000&page=21&sdate=2020-01-01&edate=2021-09-13&per=1')
text = response.text
print(text)
text_html = re.findall(',records:(.*?),pages:',text)
#html = etree.HTML(text_html)
print(text_html)



var apidata={ content:"<table class='w782 comm lsjz'><thead><tr><th class='first'>净值日期</th><th>单位净值</th><th>累计净值</th><th>日增长率</th><th>申购状态</th><th>赎回状态</th><th class='tor last'>分红送配</th></tr></thead><tbody><tr><td>2021-08-16</td><td class='tor bold'>1.0422</td><td class='tor bold'>1.0422</td><td class='tor bold red'>0.80%</td><td>场内买入</td><td>场内卖出</td><td class='red unbold'></td></tr></tbody></table>",records:414,pages:414,curpage:21};
['414']


In [62]:
def get_fund_info_per1_list(url_fund):
    # 每个基金一页一天数据，per=1，返回list
    
    list_info = [] # 存储基金信息
    
    # re.findall,xpath 匹配信息
    response = requests.get(url_fund)
    text = response.text
    text_html = re.findall('content:"(.*?)",records:',text)[0]
    html = etree.HTML(text_html)
    html_data = html.xpath('//tr/td')
    for info in html_data:
        list_info.append(info.text)  # 每个字段一行：净值日期 单位净值 累计净值 日增长率 申购状态 赎回状态 分红送配
    
    return list_info

In [ ]:
def get_timestamp(date_str):
    # 输入'2012-02-19' 得到浮点数时间戳
    return time.mktime(time.strptime(date_str,"%Y-%m-%d"))

In [93]:
def get_cnt_days(sdate,edate):  # 计算时间段间天数，截止时间是否大于起止时间
    stime = time.mktime(time.strptime(sdate,"%Y-%m-%d"))
    etime = time.mktime(time.strptime(edate,"%Y-%m-%d"))
    cnt_days = count_days = int((edate - sdate)/(24*60*60))+1
    return (cnt_days, False if cnt_days<0 else True)

In [ ]:
fund_id_list = ['512000','512880']  # 基金代码
# 每个基金的时间区间
time_interval = [['2019-01-01','2021-09-16'],
                 ['2019-01-01','2021-09-16']                
                ]

In [78]:
build_date = get_start_date('512000')
build_time_stamp = get_timestamp(build_date)

In [91]:
count_days = int((time.mktime(time.strptime('2021-09-16',"%Y-%m-%d")) - time.mktime(time.strptime('2021-09-13',"%Y-%m-%d")))/(24*60*60))
count_days

3